## Advent of Code

Day 10: Factory.

In [89]:
from icecream import ic

def factory(description: str):
    pieces = description.split(" ")

    target_lights_str = pieces[0][1:-1]
    target_lights = [light == "#" for light in target_lights_str]


    toggles_str = pieces[1:-1]
    buttons = []
    for toggle in toggles_str:
        s_cleaned = toggle[1:-1]
        str_numbers = s_cleaned.split(',')
        buttons.append([int(num) for num in str_numbers])

    target_joltages_str = pieces[-1]
    target_joltages = [int(joltage) for joltage in target_joltages_str[1:-1].split(",")]

    current_lights = [False for _ in range(len(target_lights_str))]

    return target_lights, target_joltages, current_lights, buttons


def press_button(current_lights: list, button: list):
    for light_pos in button:
        current_lights[light_pos] = not current_lights[light_pos]


def fewest(target_lights, current_lights, buttons) -> int:
    candidates = []
    already_seen = []

    if target_lights == current_lights:
        return 0

    # Seed the candidates.
    for button in buttons:
        cl = current_lights.copy()

        press_button(cl, button)

        if cl not in already_seen:
            candidates.append(cl)
            already_seen.append(cl)

    presses = 1

    while target_lights not in candidates:
        presses += 1

        new_candidates = candidates.copy()
        for candidate in candidates:

            for button in buttons:
                cl = candidate.copy()
                press_button(cl, button)

                if cl not in already_seen:
                    already_seen.append(cl)
                    new_candidates.append(cl)

        candidates = new_candidates.copy()

    return presses


target_lights, target_joltage, current_lights, buttons = factory("[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}")
ic(target_lights, target_joltage, current_lights, buttons)

assert target_lights != current_lights
press_button(current_lights, buttons[4])
press_button(current_lights, buttons[5])
assert target_lights == current_lights

target_lights, target_joltage, current_lights, buttons = factory("[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}")
found = fewest(target_lights, current_lights, buttons)
ic(found)

target_lights, target_joltage, current_lights, buttons = factory("[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}")
found = fewest(target_lights, current_lights, buttons)
ic(found)

ic| target_lights: [False, True, True, False]
    target_joltage: [3, 5, 4, 7]
    current_lights: [False, False, False, False]
    buttons: [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]]
ic| found: 2
ic| found: 3


3

In [90]:
# Part 1.

file = open("test.txt", "r")
contents = file.read()

part1 = 0
for machine_str in contents.split("\n"):
    target_lights, _, current_lights, buttons = factory(machine_str)
    presses = fewest(target_lights, current_lights, buttons)
    ic(presses)
    part1 += presses

ic(part1)

ic| presses: 2
ic| presses: 3
ic| presses: 2
ic| part1: 7


7

In [92]:
# Part 2.

import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

def fewest_joltage_presses(target_joltages: list, buttons: list) -> None|int:
    num_buttons = len(buttons)
    num_targets = len(target_joltages)

    # Matrix A has one row per equation (target_joltage),
    # and one column per variable (button).
    A = np.zeros((num_targets, num_buttons))

    for b_idx, button_targets in enumerate(buttons):
        for t_idx in button_targets:
            # button_targets contains the indices of the joltages it affects.
            if t_idx < num_targets:
                A[t_idx, b_idx] = 1

    b = np.array(target_joltages)

    # Define the objective (minimize sum: 1*b0 + 1*b1 + ...)
    c = np.ones(num_buttons)

    # Set constraints: A * x = b (so lb and ub are both 'b')
    constraints = LinearConstraint(A, lb=b, ub=b)

    # Solve for integers (integrality=1)
    res = milp(
        c=c,
        constraints=constraints,
        integrality=np.ones(num_buttons),
        bounds=Bounds(0, np.inf)
    )

    if res.success:
        # Use round() before int() to avoid rare floating-point precision issues
        # (e.g. 1.999999 -> 2)
        presses = int(np.round(res.fun))
        return presses

    else:
        ic("No integer solution found.")
    return None

_, target_joltages, _, buttons = factory("[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}")
presses = fewest_joltage_presses(target_joltages, buttons)
ic(target_joltages, buttons, presses)

file = open("input.txt", "r")
contents = file.read()

part2 = 0
for machine_str in contents.split("\n"):
    _, target_joltages, _, buttons = factory(machine_str)
    presses = fewest_joltage_presses(target_joltages, buttons)
    ic(presses)
    part2 += presses

ic(part2)

ic| target_joltages: [3, 5, 4, 7]
    buttons: [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]]
    presses: 10
ic| presses: 47
ic| presses: 36
ic| presses: 43
ic| presses: 68
ic| presses: 108
ic| presses: 28
ic| presses: 231
ic| presses: 69
ic| presses: 30
ic| presses: 254
ic| presses: 248
ic| presses: 65
ic| presses: 69
ic| presses: 61
ic| presses: 34
ic| presses: 110
ic| presses: 112
ic| presses: 111
ic| presses: 25
ic| presses: 25
ic| presses: 68
ic| presses: 68
ic| presses: 102
ic| presses: 212
ic| presses: 269
ic| presses: 65
ic| presses: 30
ic| presses: 187
ic| presses: 90
ic| presses: 246
ic| presses: 75
ic| presses: 167
ic| presses: 246
ic| presses: 138
ic| presses: 73
ic| presses: 58
ic| presses: 87
ic| presses: 75
ic| presses: 52
ic| presses: 35
ic| presses: 94
ic| presses: 200
ic| presses: 56
ic| presses: 21
ic| presses: 73
ic| presses: 84
ic| presses: 39
ic| presses: 36
ic| presses: 111
ic| presses: 173
ic| presses: 84
ic| presses: 75
ic| presses: 59
ic| presses: 46
ic| presses:

15631